# **Smartbank_text_classification**

In [ ]:
!pip install transformers datasets evaluate nltk flask Flask-Caching sentry-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from datasets import load_dataset

imdb = load_dataset("imdb")


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove stopwords
    text = ' '.join([word for word in text.lower().split() if word not in stop_words])
    return text

def tokenize_function(examples):
    examples['text'] = [preprocess_text(text) for text in examples['text']]
    return tokenizer(examples['text'], padding="max_length", truncation=True)


In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokenized_imdb = imdb.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
train_dataset = tokenized_imdb['train'].shuffle(seed=42).select(range(20000))
eval_dataset = tokenized_imdb['test'].shuffle(seed=42).select(range(5000))


In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./result',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


In [ ]:
import numpy as np
import evaluate
from sklearn.metrics import precision_recall_fscore_support

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = metric.compute(predictions=predictions, references=labels)
    return {
        'accuracy': acc['accuracy'],
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)
trainer.train()
results = trainer.evaluate()
print(results)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,0.691800
20,0.705200
30,0.696300
40,0.686500
50,0.679700
60,0.684000
70,0.675700
80,0.658600
90,0.635600
100,0.636900


{'eval_loss': 0.38729920983314514, 'eval_accuracy': 0.9028, 'eval_precision': 0.8998416468725258, 'eval_recall': 0.907023144453312, 'eval_f1': 0.9034181240063593, 'eval_runtime': 84.904, 'eval_samples_per_second': 58.89, 'eval_steps_per_second': 0.93, 'epoch': 3.0}


In [ ]:
model.config.id2label = {0: "NEGATIVE", 1: "POSITIVE"}
model.config.label2id = {"NEGATIVE": 0, "POSITIVE": 1}
model.save_pretrained('smart_sentiment_analysis')
tokenizer.save_pretrained('smart_sentiment_analysis')


('smart_sentiment_analysis/tokenizer_config.json',
 'smart_sentiment_analysis/special_tokens_map.json',
 'smart_sentiment_analysis/vocab.txt',
 'smart_sentiment_analysis/added_tokens.json')

In [ ]:
from huggingface_hub import login
login()


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained('./smart_sentiment_analysis')
tokenizer = AutoTokenizer.from_pretrained('./smart_sentiment_analysis')

model.push_to_hub("smart_sentiment_analysis")
tokenizer.push_to_hub("smart_sentiment_analysis")


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Beehzod/smart_sentiment_analysis/commit/d6ed85137045b7a3de0971c60a761206f22b9226', commit_message='Upload tokenizer', commit_description='', oid='d6ed85137045b7a3de0971c60a761206f22b9226', pr_url=None, pr_revision=None, pr_num=None)

# **test models**

In [ ]:
from transformers import pipeline


sentiment_pipeline = pipeline("sentiment-analysis", model="Beehzod/smart_sentiment_analysis")


text = (["This movie was fantastic! I really enjoyed it","I hate  doing examples"])
results = sentiment_pipeline(text)

for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.4f}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Label: POSITIVE, Score: 0.9956
Label: NEGATIVE, Score: 0.6704


In [ ]:
from transformers import pipeline

# Load the model from the Hugging Face Hub
ner_pipeline = pipeline("ner", model="Beehzod/smart-finetuned-ner")

# Example predictions
text = "Hugging Face Inc. is based in New York City, and its CEO is Clement Delangue."
results = ner_pipeline(text)

for entity in results:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Entity: Hu, Label: B-ORG, Score: 0.9908
Entity: ##gging, Label: I-ORG, Score: 0.9968
Entity: Face, Label: I-ORG, Score: 0.9990
Entity: Inc, Label: I-ORG, Score: 0.9988
Entity: ., Label: I-ORG, Score: 0.9969
Entity: New, Label: B-LOC, Score: 0.9989
Entity: York, Label: I-LOC, Score: 0.9986
Entity: City, Label: I-LOC, Score: 0.9984
Entity: Clement, Label: B-PER, Score: 0.9990
Entity: Del, Label: I-PER, Score: 0.9994
Entity: ##ang, Label: I-PER, Score: 0.9988
Entity: ##ue, Label: I-PER, Score: 0.9987
